**Short answers**

* **Do I need to extract features?**
  Not required. A good **TF‑IDF** (or BM25) on the **raw concatenated text** works; **adding structured attrs** (color, fabric, brand) as tokens **helps** (esp. for exact filters + cold start).

* **Better approach for “similar products”?**
  Use **sentence embeddings** (multilingual) for true semantic similarity; optionally **hybrid** = embeddings (or TF‑IDF) for recall → **rerank** with catalog constraints (same category/price) and/or **co‑purchase**.

---

### Minimal pipelines

**A) TF‑IDF (fast, strong baseline)**

```python
# text field: title + desc + attrs (weighted by duplication or prefixes)
def build_text(row):
    return f"title: {row.title} brand: {row.brand} material: {row.material} desc: {row.desc}"

corpus = df.apply(build_text, axis=1)

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(analyzer='word',
                      ngram_range=(1,2),
                      min_df=5, max_df=0.6,
                      stop_words='swedish')  # or None + char_wb 3–5 for typos
X = vec.fit_transform(corpus)                   # items×vocab (L2 normalized)

# query vector & nearest neighbors
qv = vec.transform([build_text(query_item_row)])
scores = (X @ qv.T).A.ravel()
idx = scores.argsort()[::-1]
candidates = df.iloc[idx].head(50)             # filter: same category, price window, etc.
```

**B) Sentence embeddings (better semantics, multilingual)**

```python
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer
enc = SentenceTransformer("multilingual-MiniLM")  # pick a small multilingual model
E = enc.encode(corpus, normalize_embeddings=True) # items×d

# cosine similarity
import numpy as np
q = enc.encode([build_text(query_item_row)], normalize_embeddings=True)[0]
scores = (E @ q)                                  # dot = cosine
idx = np.argsort(-scores)
candidates = df.iloc[idx].head(50)
```

**Hybrid idea:** `score = 0.7 * cosine_embed + 0.3 * cosine_tfidf`, then apply **category filter** and business rules.

---

### What you might be missing

* **Language-specific normalization** (Swedish): lowercasing, Unicode normalization, optional **stemming/lemmatization** (Snowball “swedish”), keep numerics/units (“100% bomull”, “40°”).
* **Synonym/normalization table** (e.g., “bomull”→“cotton”, “pyjamas/pajamas”).
* **Field weighting**: upweight title/brand (duplicate tokens or separate vectorizers and sum).
* **ANN index** (FAISS/Annoy) for fast retrieval at scale.
* **Blending with behavior** (co‑purchase) if you also want complements.

---

### Offline metrics (use what you have)

* **Recall\@K / HitRate\@K / NDCG\@K** using proxy positives: same **category/brand**, or **co-view/co-purchase substitutes**.
* **Category purity** of k-NN (percentage of neighbors in same category).
* **Coverage** (items with ≥1 neighbor), **diversity** (brand/category spread).
* If you have clicks: **MRR/NDCG\@K** against clicked‑next similar items.

**Tip:** even if you don’t extract columns, **include known attrs into the text** (e.g., `color=blå material=bomull`) so both TF‑IDF and embeddings “see” them.


Color/category filters at retrieval time; boost rather than hard‑filter if recall drops.
Hybrid with CF: final score = 0.7*embed_cos + 0.3*CF_sim when an anchor group (from cart/ PDP) is available.
Offline eval: category/brand/color purity@K, Recall/HitRate/NDCG@K vs. proxy “similar” pairs (co‑view/co‑purchase in same category), coverage and diversity.


# Build in-domain positives from your data (example: same-category co-purchase pairs)
pos_pairs = [(gid_a, gid_b) for (gid_a,gid_b) in co_purchase_pairs if same_category(gid_a,gid_b)]
gid2text = dict(zip(group_df.groupId, group_df.text))

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

train_ex = [InputExample(texts=[gid2text[a], gid2text[b]]) for a,b in pos_pairs]
loader   = DataLoader(train_ex, batch_size=64, shuffle=True)

model = SentenceTransformer("KBLab/sentence-bert-swedish-cased", device="cpu")
loss  = losses.MultipleNegativesRankingLoss(model)   # contrastive retrieval loss

model.fit(train_objectives=[(loader, loss)], epochs=1)  # 1–3 epochs is often enough

# Encode & search as before; keep your soft rerank (name/brand/color boosts)
E = model.encode(group_df["text"].tolist(), normalize_embeddings=True, batch_size=64).astype("float32")


## Assemble a DS

In [1]:
import pandas as pd

art = pd.read_csv(
    "../../data/processed/articles_clean.csv",
    usecols=["sku","groupId", "name", "brand", "description", "audience", "category", "priceSEK", "color", "fabrics"],
    dtype=str
)

In [2]:
# Remove articles with priceSEK below 1 from the DataFrame and convert to numeric
art['priceSEK'] = pd.to_numeric(art['priceSEK'], errors='coerce')
art = art[art['priceSEK'] >= 1]

In [3]:
# Create price buckets based on the distribution of price_sek, using 6 buckets
price_bins = [0, 100, 300, 600, 1000, 2000, float('inf')]
price_labels = [
    'Budget',        # 0-100
    'Value',         # 100-300
    'Popular',       # 300-600
    'Premium',       # 600-1000
    'Luxury',        # 1000-2000
    'Exclusive'      # 2000+
]
art['priceband'] = pd.cut(art['priceSEK'], bins=price_bins, labels=price_labels, include_lowest=True)
art['priceband'] = art['priceband'].astype(object)
art.loc[art['priceSEK'].isna(), 'priceband'] = 'unknown'

In [4]:
with pd.option_context('display.max_columns', None, 'display.width', 0):
    display(art.sort_values("groupId"))

,sku,groupId,name,description,brand,color,audience,category,priceSEK,fabrics,priceband
11331,055522,055522,Tröja,Sticka en färgglad och trendig tröja i garnet ...,Gjestal Garn,NaN,dam,Tröjor,29.0,NaN,Budget
11330,055573,055573,Luva,Sticka en trendig huva i garnet Halaus från No...,Novita,vit,dam,"Mössor & hattar,Mönster",29.0,NaN,Budget
11329,055575,055575,Vantar,Sticka ett par vantar med blomstermotiv i garn...,Novita,vit,dam,Vantar,29.0,NaN,Budget
11328,055576,055576,Benvärmare,Sticka ett par trendiga benvärmare i garnet Ha...,Novita,vit,dam,Sockor & strumpor,29.0,NaN,Budget
11315,095318,095302,Garn Drops Nepal,Garn Drops Nepal är ett underbart tjockt luxuö...,Drops Design,svart,unknown,unknown,33.0,"['alpaca', 'ull']",Budget
...,...,...,...,...,...,...,...,...,...,...,...
14,AH2031-4547,AH2031,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
19,AH2031-4244,AH2031,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
20,AH3021-4244,AH3021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
21,AH3021-4547,AH3021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value


In [5]:
# Deduplicate so that for each groupId, keep the first row for all columns except 'color', 
# which should be a list of all colors from merged rows (excluding missing/unknown/nan/none).
def merge_colors(series):
    # Remove missing/unknown/nan/none and deduplicate
    colors = [str(c).strip() for c in series if pd.notna(c) and str(c).strip().lower() not in {"", "unknown", "nan", "none"}]
    return list(sorted(set(colors))) if colors else []

art = art.sort_values("sku")  # Ensure deterministic "first" row
art = art.groupby("groupId", as_index=False).agg(
    {col: (merge_colors if col == "color" else "first") for col in art.columns if col != "sku"}
)


In [6]:
art

,groupId,name,description,brand,color,audience,category,priceSEK,fabrics,priceband
0,055522,Tröja,Sticka en färgglad och trendig tröja i garnet ...,Gjestal Garn,[],dam,Tröjor,29.0,None,Budget
1,055573,Luva,Sticka en trendig huva i garnet Halaus från No...,Novita,[vit],dam,"Mössor & hattar,Mönster",29.0,None,Budget
2,055575,Vantar,Sticka ett par vantar med blomstermotiv i garn...,Novita,[vit],dam,Vantar,29.0,None,Budget
3,055576,Benvärmare,Sticka ett par trendiga benvärmare i garnet Ha...,Novita,[vit],dam,Sockor & strumpor,29.0,None,Budget
4,095302,Garn Drops Nepal,Garn Drops Nepal är ett underbart tjockt luxuö...,Drops Design,"[beige, blå, blå-grön, brun, grå, gråmelerad, ...",unknown,unknown,33.0,"['alpaca', 'ull']",Budget
...,...,...,...,...,...,...,...,...,...,...
1568,AH1021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,[vit-beige],generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
1569,AH2021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,[svart-grå],generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
1570,AH2031,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,[svart-grå],generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
1571,AH3021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,[blå],generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value


## 1. Build a clean text field for vectorization

In [7]:
import pandas as pd, numpy as np, unicodedata, re, ast

MISSING = {"", "unknown", "nan", "none", None}

# --- tiny helpers ---
def canon(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"\u00A0", " ", s)                       # nbsp
    s = re.sub(r"[\u2010-\u2015\u2212\-]+", "-", s)     # dashes
    s = re.sub(r"\s+", " ", s).strip()
    return s

def as_list(x):
    if isinstance(x, (list, tuple, np.ndarray, pd.Series)): return [str(v) for v in x if str(v).strip() not in MISSING]
    if str(x).strip() in MISSING: return []
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try: return [str(v) for v in ast.literal_eval(s)]
        except Exception: return [s]
    return [s]

# Just use the colors as they appear in the color column, splitting on common delimiters and lowercasing
def norm_colors(x):
    toks = []
    for c in as_list(x):
        c = c.lower().strip()
        # Split on common delimiters: comma, slash, whitespace, dash
        parts = re.split(r"[/,\s]+|-", c)
        for p in parts:
            p = p.strip()
            if p and p not in MISSING:
                toks.append(p)
    # dedup preserve order
    seen, out = set(), []
    for t in toks:
        if t and t not in seen:
            seen.add(t); out.append(t)
    return out

# No normalization for materials, just as_list and dedup (preserve order)
def norm_materials(x):
    seen, out = set(), []
    for m in as_list(x):
        t = m.strip()
        if t and t not in seen:
            seen.add(t); out.append(t)
    return out

def norm_categories(x):
    cats = [canon(c) for c in str(x).split(",") if str(c).strip() not in MISSING]
    # dedup preserve order
    seen, out = set(), []
    for c in cats:
        if c and c.lower() not in seen:
            seen.add(c.lower()); out.append(c)
    return out

# --- build text per groupId (natural sentences for embeddings) ---
art = art.copy()
art["colors_norm"]    = art["color"].apply(norm_colors)
art["materials_norm"] = art["fabrics"].apply(norm_materials)
art["categories_norm"]= art["category"].apply(norm_categories)

def build_text_embed(r):
    name = canon(r.get("name",""))
    desc = canon(r.get("description",""))[:800]  # keep units/%/°
    brand = canon(r.get("brand","")) if r.get("brand","") not in MISSING else ""
    audience = r.get("audience","")
    audience = audience if str(audience).lower() not in MISSING and audience!="generic" else ""
    priceband = r.get("priceband","")
    priceband = priceband if str(priceband).lower() not in MISSING else ""
    price = r.get("priceSEK", "")
    cats = r.get("categories_norm", [])
    cols = r.get("colors_norm", [])
    mats = r.get("materials_norm", [])

    parts = []
    if name: parts.append(f"{name}.")
    if desc: parts.append(desc)
    meta = []
    if brand:    meta.append(f"Varumärke: {brand}.")
    if cats:     meta.append("Kategori: " + ", ".join(cats) + ".")
    if audience: meta.append(f"Målgrupp: {audience}.")
    if mats:     meta.append("Material: " + ", ".join(mats) + ".")
    if cols:     meta.append("Färger: " + ", ".join(cols) + ".")
    if pd.notna(price) and str(price).strip() not in MISSING:
        meta.append(f"Pris: {int(round(float(price)))} SEK.")
    if priceband: meta.append(f"Prisnivå: {priceband}.")
    return " ".join(parts + meta).strip()

art["text"] = art.apply(build_text_embed, axis=1)
group_df = art.loc[
    art["text"].ne(""),
    ["groupId","text","colors_norm","materials_norm","categories_norm","brand","priceband"]
].reset_index(drop=True)
corpus = group_df["text"].tolist()

In [8]:
pd.set_option('display.max_colwidth', None)
group_df.head()

,groupId,text,colors_norm,materials_norm,categories_norm,brand,priceband
0,055522,Tröja. Sticka en färgglad och trendig tröja i garnet Vera från House of Yarn! Rekommenderade stickor: Stor och liten rundsticka och strumpsticka 6 och 8 mm Varumärke: Gjestal Garn. Kategori: Tröjor. Målgrupp: dam. Material: None. Pris: 29 SEK. Prisnivå: Budget.,[],[None],[Tröjor],Gjestal Garn,Budget
1,055573,"Luva. Sticka en trendig huva i garnet Halaus från Novita! Rekommenderade stickor: Rundsticka 40 och 80 cm, 10 mm Varumärke: Novita. Kategori: Mössor & hattar, Mönster. Målgrupp: dam. Material: None. Färger: vit. Pris: 29 SEK. Prisnivå: Budget.",[vit],[None],"[Mössor & hattar, Mönster]",Novita,Budget
2,055575,Vantar. Sticka ett par vantar med blomstermotiv i garnet Halaus från Novita! Rekommenderade stickor: Strumpstickor 7mm Varumärke: Novita. Kategori: Vantar. Målgrupp: dam. Material: None. Färger: vit. Pris: 29 SEK. Prisnivå: Budget.,[vit],[None],[Vantar],Novita,Budget
3,055576,Benvärmare. Sticka ett par trendiga benvärmare i garnet Halaus från Novita! Rekommenderade stickor: Strumpstickor 8 och 10 mm Varumärke: Novita. Kategori: Sockor & strumpor. Målgrupp: dam. Material: None. Färger: vit. Pris: 29 SEK. Prisnivå: Budget.,[vit],[None],[Sockor & strumpor],Novita,Budget
4,095302,"Garn Drops Nepal. Garn Drops Nepal är ett underbart tjockt luxuöst garn spunnet av 35% superfin alpaca och 65 % peruvian highland ull. Garnfibrerna är obehandlade, vilket innebär att de enbart har tvättats och inte utsatts för kemisk behandling innan färgning. Detta lyfter fram garnet naturliga egenskaper på bästa sätt och ger bättre form och textur.Blandningen lyfter fram alpacans sidenmjuka yta, samtidigt som ullen bidrar till bättre form och stabilitet. Garnet är spunnet av 3 trådar vilket ger en spännande, rustik och vacker maskbild. Garn Drops Nepal är en lättstickad/virkad kvalitet som också passar utmärkt att tova. Kvalité: 65% ull och 35% alpaca.Vikt/längd: 50 gram &#61; ca 75 m.Rekommenderade stickor: 5 mm.Stickfasthet: 10 x 10 cm &#61; 17 m x 22 v.Tvättråd: Handtvätt, max 30°. Varumärke: Drops Design. Material: alpaca, ull. Färger: beige, blå, grön, brun, grå, gråmelerad, gul, lila, marin, mörkgrå, orange, rosa, röd, svart, turkos, vit. Pris: 33 SEK. Prisnivå: Budget.","[beige, blå, grön, brun, grå, gråmelerad, gul, lila, marin, mörkgrå, orange, rosa, röd, svart, turkos, vit]","[alpaca, ull]",[],Drops Design,Budget


In [9]:
# === TF‑IDF (word bigrams + char 3–5 for robustness) + cosine KNN ===
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize, MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack, csr_matrix
import numpy as np
import pandas as pd
import re

# --- build text features (same as you had) ---
group_df = group_df.reset_index(drop=True)
corpus   = group_df["text"].astype(str).tolist()
gids     = group_df["groupId"].to_numpy()
colors   = group_df["colors_norm"].apply(lambda x: set(x or [])).to_numpy()
cats_col = group_df["categories_norm"].apply(lambda x: x or []).tolist()

vec_w = TfidfVectorizer(analyzer="word", ngram_range=(1,2), min_df=2, max_df=0.8)
vec_c = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5), min_df=3)
Xw = vec_w.fit_transform(corpus)
Xc = vec_c.fit_transform(corpus)

# --- category block as multi-hot (sparse) ---
mlb = MultiLabelBinarizer(sparse_output=True)
C = mlb.fit_transform(cats_col)           # shape: [n_docs, n_cats]
C = csr_matrix(C)                         # ensure CSR for ops

# Row-normalize each block so weights are comparable, then apply block weights
Xw_n = normalize(Xw)        # word TF-IDF
Xc_n = normalize(Xc)        # char TF-IDF
C_n  = normalize(C)         # categories (binary)

W_WORD, W_CHAR, W_CAT = 1.0, 1.0, 3.0     # <= raise W_CAT to prioritize categories more
X = normalize(hstack([W_WORD*Xw_n, W_CHAR*Xc_n, W_CAT*C_n]).tocsr())

# KNN over the fused space
POOL = min(201, len(group_df))
nn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=POOL).fit(X)


# --- search helpers ---
def search_by_gid(gid, k=50, require_color=None):
    i = int(np.where(gids == gid)[0][0])
    d, idx = nn.kneighbors(X[i], n_neighbors=min(k+1, len(gids)))  # +1 for self
    idx, d = idx[0], d[0]
    m = idx != i
    idx, d = idx[m], d[m]
    if require_color:
        m2 = np.array([require_color in colors[j] for j in idx], bool)
        idx, d = idx[m2], d[m2]
    sim = 1.0 - d
    return group_df.iloc[idx].assign(similarity=sim).head(k)

def search_by_text(q, k=50, require_color=None):
    qv = normalize(hstack([vec_w.transform([q]), vec_c.transform([q])]))
    d, idx = nn.kneighbors(qv, n_neighbors=min(k, len(gids)))
    idx, d = idx[0], d[0]
    if require_color:
        m2 = np.array([require_color in colors[j] for j in idx], bool)
        idx, d = idx[m2], d[m2]
    sim = 1.0 - d
    return group_df.iloc[idx].assign(similarity=sim).head(k)


In [10]:
# Build neighbors for all items (self + 10)
POOL = min(11, len(group_df))
dists, idxs = nn.kneighbors(X, n_neighbors=POOL)

gids = group_df["groupId"].to_numpy()
src  = np.repeat(gids, POOL)
rec  = gids[idxs.ravel()]
sim  = 1.0 - dists.ravel()  # cosine similarity

recs = pd.DataFrame({"src_groupId": src, "rec_groupId": rec, "similarity": sim})
recs = recs[recs["src_groupId"] != recs["rec_groupId"]].copy()

# keep top-10 per source
recs["rec_rank"] = (
    recs.groupby("src_groupId")["similarity"].rank(method="first", ascending=False).astype(int)
)
recs = recs[recs["rec_rank"] <= 10].sort_values(["src_groupId","rec_rank"])

# similarity cutoff — calibrate by quantile (e.g., keep top 80% most similar)
SIM_CUT = recs["similarity"].quantile(0.01)  # keep top 99,9% (adjust as needed)
recs = recs[recs["similarity"] >= SIM_CUT].copy()

# attach metadata for review
meta_cols = [c for c in ["text","brand","categories_norm","colors_norm","priceband"]
             if c in group_df.columns]
meta = group_df.set_index("groupId")[meta_cols]
recs = recs.join(meta.add_prefix("src_"), on="src_groupId") \
           .join(meta.add_prefix("rec_"), on="rec_groupId")

recs.to_csv("tfidf_similar_items_top10.csv", index=False)


In [11]:
# Find src_groupId that have less than one rec (i.e., zero recs)
src_counts = recs["src_groupId"].value_counts()
no_rec_srcs = src_counts[src_counts < 3]
print("src_groupId with less than one rec:", no_rec_srcs.index.tolist())


src_groupId with less than one rec: ['559097', '360140', '305043', '549005', '549183']


In [14]:
recs[recs["src_groupId"] == "260406"]

,src_groupId,rec_groupId,similarity,rec_rank,src_text,src_brand,src_categories_norm,src_colors_norm,src_priceband,rec_text,rec_brand,rec_categories_norm,rec_colors_norm,rec_priceband
2707,260406,264804,0.818391,1,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Framknäppt mjuk bh i bomull. B &#61; BCD, E &#61; EF, G&#61;GH Skön framknäppt bomulls-bh från Glamoris med rygg i genombruten spets som andas. Släta kupor med stretch, anpassar formen. T-rygg håller de mjuka, sömlösa axelbanden på plats. Utformad för komfort - både för dagen och på natten. 60% bomull, 30% polyamid, 10% elastan. Fintvätt 40°. Varumärke: Glamorise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Underkläder. Målgrupp: dam. Material: bomull, elastan. Färger: beige, rosa, vit. Pris: 439 SEK. Prisnivå: Popular.",Glamorise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Underkläder]","[beige, rosa, vit]",Popular
2708,260406,261324,0.800380,2,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Bh topp Heidi. Heidi en mjuk och skön bh-topp i trikåmaterial frånTrofé. Bh:n är utan sömmar och knäppning. Bh:n dras över huvudet och har inga skavande detaljer eller byglar vilket också gör den utmärkt att sova i. Kuporna är vadderade men vadderingen går att plocka bort. Bh:n har brett resår nertill och breda axelband som ger extra komfort. Rynkeffekt mellan brösten. 90% nylon, 10% elastan. Handtvätt. Varumärke: Trofé. Kategori: Bh utan bygel, Bh-toppar, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: nylon, elastan. Färger: svart, vit. Pris: 179 SEK. Prisnivå: Value.",Trofé,"[Bh utan bygel, Bh-toppar, Bh, Bh utan kupstorlek, Underkläder]","[svart, vit]",Value
2709,260406,260423,0.757057,3,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Framknäppt bh. Formsydd framknäppt bh som sitter fantastisk skönt och bekvämt. Brett ryggparti som ger lite stöd och slätar ut. Reglerbara och mjuka axelband. 90% polyamid, 10% elastan. Fintvätt 30°. Varumärke: Louise. Kategori: Bh utan bygel, Framknäppt bh, Bh, Underkläder. Målgrupp: dam. Material: elastan. Färger: vit. Pris: 229 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Framknäppt bh, Bh, Underkläder]",[vit],Value
2710,260406,260372,0.756245,4,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett rygg

## 2. Vectorize with TF-IDF (Term Frequency × Inverse Document Frequency)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2), #use unigrams + bigrams
    min_df=1,
    strip_accents=None,   # keep å/ä/ö
    # sublinear_tf=True,  # optional
    # dtype=np.float32,   # optional memory saver
    token_pattern=r'(?u)\b\w+\b'
)
X_tfidf = tfidf.fit_transform(group_df["text"])
X_tfidf.shape

(8362, 92930)

## 3. Singular Value Decomposition + L2 normalize

TF-IDF gives precise but sparse signals; SVD compresses & generalizes them.
L2-norm makes nearest-neighbor search stable and comparable across items.

In [27]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

n_components = min(128, max(2, X_tfidf.shape[1]-1)) 
svd = TruncatedSVD(n_components=n_components, random_state=0)
X_svd = svd.fit_transform(X_tfidf)
X_emb = normalize(X_svd)
X_emb.shape


(8362, 128)

## 4. Build 10-nearest neighbors (cosine) and return a small recs table


* It’s the **cosine of the angle** between vectors $a$ and $b$:

  $$
  \text{cosine\_sim}(a,b)=\frac{a\cdot b}{\|a\|\;\|b\|}
  $$

  * $=1$ → same direction (very similar)
  * $=0$ → orthogonal (unrelated)
  * $=-1$ → opposite (rare with TF-IDF since values are ≥0)

* After we **L2-normalize** vectors, cosine similarity becomes just the **dot product**.

* In scikit-learn, `metric="cosine"` actually computes **cosine distance**:

  $$
  \text{cosine\_dist} = 1 - \text{cosine\_sim}
  $$

  That’s why in the code we convert back with `similarity = 1 - d`.

Why we use it: it’s **scale-invariant** (ignores length), so two SKUs with similar wording but different text lengths still match well.


In [28]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

POOL = min(11, len(group_df))
nn = NearestNeighbors(metric="cosine", n_neighbors=POOL).fit(X_emb)
dists, idxs = nn.kneighbors(X_emb)

groupid_arr = group_df["groupId"].values

# Build all recommendations, then filter out self-recommendations
src_groupids = np.repeat(groupid_arr, POOL)
rec_groupids = groupid_arr[idxs.ravel()]
similarities = 1 - dists.ravel()

recs = pd.DataFrame({
    "src_groupId": src_groupids,
    "rec_groupId": rec_groupids,
    "similarity": similarities
})

# Remove self-recommendations
recs = recs[recs["src_groupId"] != recs["rec_groupId"]].copy()

# For each src_groupId, keep only the top POOL-1 recommendations (in case of ties or accidental duplicates)
recs["rec_rank"] = (
    recs.groupby("src_groupId")["similarity"]
    .rank(method="first", ascending=False)
    .astype(int)
)
recs = recs[recs["rec_rank"] <= POOL-1]

recs.head(10)

,src_groupId,rec_groupId,similarity,rec_rank
1,261290,261375,0.975085,1
2,261290,261296,0.970066,2
3,261290,261371,0.964389,3
4,261290,261002,0.961406,4
5,261290,261649,0.954276,5
6,261290,266882,0.948855,6
7,261290,261221,0.948009,7
8,261290,261561,0.945863,8
9,261290,261228,0.939821,9
10,261290,260878,0.936348,10


In [29]:
recs.sample(15, random_state=42)

,src_groupId,rec_groupId,similarity,rec_rank
60124,200266,200198,0.679553,9
36425,250040,250057,0.967614,4
77632,251934,250004,0.922400,5
63687,210340,210080,0.867983,8
6807,264006,103501,0.927784,9
24499,552955,552911,0.993026,2
10210,262881,260215,0.902545,2
41346,400646,400641,0.992933,8
76359,261271,263178,0.847963,8
50860,598050,520076,0.851846,7


In [30]:
target = "190041"  # example groupId

# pick columns and collapse art to one row per groupId
keep = [c for c in ["groupId","name","name.1","color","audience","category","priceband","description"] if c in art.columns]
details = art.drop_duplicates("groupId")[keep].copy()

view = (
    recs.loc[recs["src_groupId"].eq(target), ["rec_groupId","rec_rank","similarity"]]
        .merge(details, left_on="rec_groupId", right_on="groupId", how="left")
        .drop(columns=["groupId"])  # joined key
        [["rec_rank","similarity","rec_groupId"] + [c for c in keep if c != "groupId"]]
        .sort_values("rec_rank")
        .reset_index(drop=True)
)

view


,rec_rank,similarity,rec_groupId,name,name.1,audience,category,priceband,description
0,1,0.958458,500355,Madrasskydd i 6 storlekar,Linea,hemmet,"Bädd,Bäddtillbehör,Bädd (linea)",Value,Madrasskydd i en tålig bomull- och polyesterblandning. Madrasskyddet är quiltat och formsytt samt hålls på plats med hjälp av resårband på kortsidorna. Finns i flera olika storlekar.Material
1,2,0.956018,579830,Kanallängd Malena,Linea,hemmet,"Kanallängder,Gardiner (linea)",Popular,Malena kanallängd är snygga längder i ett av våra mest populära mönster som finns i flera olika färgkombinationer.Material
2,3,0.954849,521879,Innerkudde Rund,Linea,hemmet,"Innerkuddar,Bädd (linea)",Value,"Rund innerkudde med ett yttertyg av bomull och polyester, fyllning av polyester. Kudden är Oeko-Tex certifierad vilket är ett intyg på att den inte orsakar hälsoproblem. Storlek"
3,4,0.952103,585060,Innertäcke Medium,Linea,hemmet,"Täcken,Bädd (linea)",Popular,"Mjukt quiltstickat innertäcke med ett elegant yttertyg av vävd kvalitet. Ett bandkant går längst med hela yttertyget. Fyllningen är gjord av silikoniserad polyester, hålfiber.Vikt"
4,5,0.952103,585061,Innertäcke Varm,Linea,hemmet,"Täcken,Bädd (linea)",Popular,"Mjukt quiltstickat innertäcke med ett elegant yttertyg av vävd kvalitet. Ett bandkant går längst med hela yttertyget. Fyllningen är gjord av silikoniserad polyester, hålfiber.Vikt"
5,6,0.950776,551100,Bordsduk Sara blommig,Linea,hemmet,Dukar,Value,Underbar duk med ett tryckt blommotiv i härliga pastellfärger. Ett enkelt sätt att göra din dukning lite extra fin.
6,7,0.949392,579009,Täcke Kingsize,Linea,hemmet,"Täcken,Bädd (linea)",Popular,Mjukt och behagligt quiltat täcke med bandkant. Yttertyget är av tätvävd polyester och fyllning av silikoniserad polyester. Hålfiber 800g. Storlek
7,8,0.945042,537138,Kanallängd Ingrid,Linea,hemmet,Kanallängder,Premium,Klassiska gardinlängder med ett garnfärgat rutmönster i vackra färgkombinationer .Då väven är garnfärgad blir längderna lika fin på båda sidor. Rutiga omtag 7x70cm medföljer.Material
8,9,0.942633,525001,Blommig Multibandslängd Sammet,Linea,unknown,Multibandslängder,Popular,Exlusiva sammetsgardiner med digitaltryckt motiv i underbara färger. Multibandet på baksidan har flera upphängningsfunktioner.Storlek
9,10,0.941961,541012,Bågkappa med brodyr Lavendel,Linea,hemmet,"Gardinbåge,Gardiner (linea)",Value,Underbart vacker gardinbåge med exklusiv brodyr och spetsdetaljer i lantlig romantisk stil. Kombinera gärna med flera av våra produkter i samma serie för att få en komplett look.Storlek


In [31]:
# Visualize the recommendations for the target itself (i.e., show the row for the target)
cols = ["rec_rank", "similarity", "rec_groupId"] + [c for c in keep if c != "groupId"]
if "description" not in cols:
    cols.append("description")
view_target = (
    details.loc[details["groupId"] == target]
        .assign(rec_rank=1, similarity=1.0, rec_groupId=target)
        [cols]
        .reset_index(drop=True)
)
view_target


,rec_rank,similarity,rec_groupId,name,name.1,audience,category,priceband,description
0,1,1.0,190041,Innerkudde,Linea,hemmet,"Kuddar,Innerkuddar,Bädd (linea)",Value,Fyllnadskudde till kuddfodral med ett yttertyg av polyester/bomull och fyllning av polyesterfibrer. Innerkudden finns i flera olika storlekar för att matcha alla våra olika kuddfodral. Färg


In [32]:
out = (recs
       .sort_values(['src_groupId','rec_rank'])
       .groupby('src_groupId')['rec_groupId']
       .apply(list)
       .reset_index(name='recs'))

out.to_csv("../data/predictions/vector_similarity_recommendations.csv", index=False)


In [33]:
out

,src_groupId,recs
0,0044,"[870023, 262790, 262212, 261266, 261024, 261784, 261172, 261966, 261271, 352301]"
1,052743,"[520768, 582111, 569744, 569694, 582109, 528417, 562744, 522383, 522391, 581173]"
2,055522,"[170005, 273763, 261883, 261891, 273755, 261917, 261909, 273771, 273789, 210375]"
3,055573,"[055575, 055576, 273821, 205948, 525933, 205955, 204057, 204032, 202762, 201244]"
4,055575,"[055576, 207027, 055573, 319525, 509596, 273821, 294819, 516849, 203349, 203331]"
...,...,...
8357,AH2021,"[AH2031, AH1021, AH3021, 266460, 266486, 266445, 266312, 266346, 266253, 266338]"
8358,AH2031,"[AH2021, AH1021, 266460, AH3021, 266486, 266445, 266312, 266346, 266338, 266253]"
8359,AH3021,"[AH2021, AH1021, AH2031, 266460, 266486, 266445, 266312, 266253, 266346, 266338]"
8360,SOFIND,"[270140, 490082, 508921, 395702, 303030, 361065, 395713, 459978, 536755, 490083]"
